<a href="https://www.kaggle.com/code/annaselinabaniakina/digit-recognition?scriptVersionId=200652963" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
#load datasets
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
#reshape and normalize training data
X_train = train_data.drop(columns='label').values.reshape(-1, 28, 28, 1) / 255.0
y_train = train_data['label']

#normalize test data
test_data = test_data.values.reshape(-1, 28, 28, 1) / 255.0

In [4]:
from tensorflow.keras import layers, models

#build a deeper CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  #dropout to reduce overfitting
    layers.Dense(10, activation='softmax')  #output layer for 10 classes
])

#compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

2024-10-12 19:40:38.951983: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 19:40:38.952134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 19:40:39.110811: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

#early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#train the model
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
525/525 [==============================] - 19s 33ms/step - loss: 0.4431 - accuracy: 0.8591 - val_loss: 0.1129 - val_accuracy: 0.9636
Epoch 2/30
525/525 [==============================] - 16s 31ms/step - loss: 0.1385 - accuracy: 0.9593 - val_loss: 0.0701 - val_accuracy: 0.9781
Epoch 3/30
525/525 [==============================] - 17s 31ms/step - loss: 0.0987 - accuracy: 0.9715 - val_loss: 0.0666 - val_accuracy: 0.9801
Epoch 4/30
525/525 [==============================] - 16s 30ms/step - loss: 0.0784 - accuracy: 0.9776 - val_loss: 0.0565 - val_accuracy: 0.9815
Epoch 5/30
525/525 [==============================] - 17s 32ms/step - loss: 0.0642 - accuracy: 0.9804 - val_loss: 0.0507 - val_accuracy: 0.9844
Epoch 6/30
525/525 [==============================] - 16s 31ms/step - loss: 0.0540 - accuracy: 0.9828 - val_loss: 0.0528 - val_accuracy: 0.9836
Epoch 7/30
525/525 [==============================] - 16s 31ms/step - loss: 0.0465 - accuracy: 0.9859 - val_loss: 0.0566 - val_accuracy:

In [6]:
val_loss, val_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.9895


In [7]:
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

875/875 [==============================] - 5s 6ms/step


In [8]:
submission = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_labels) + 1),
    'Label': predicted_labels
})

#save the submission file
submission.to_csv('/kaggle/working/submission.csv', index=False)